In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

from collections import OrderedDict
from random import randint
import random
import decimal
from operator import add

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
max_hidden_layers = 9
max_neurons_per_layer = 100
min_learning_rate = .0001
max_learning_rate = .01
acti_func = ["softmax", "elu", "selu", "softplus", "softsign", 
             "relu", "tanh", "sigmoid", "hard_sigmoid", "linear"]
num_individuals = 16
loss_weight = 10
accuracy_weight = 10
time_weight = 10

In [3]:
def load_data():
    iris_data = load_iris()
    print('Example Data:')
    print(iris_data.data[:5])
    print('Example Labels:')
    print(iris_data.target[:5])
    global iris_data

<ipython-input-3-11c664384486>:7: SyntaxWarning: name 'iris_data' is assigned to before global declaration
  global iris_data


In [4]:
def process_data():
    x = iris_data.data
    y_ = iris_data.target.reshape(-1,1)

    encoder = OneHotEncoder(sparse=False)
    y = encoder.fit_transform(y_)
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=.20)
    global train_x, test_x, train_y, test_y

<ipython-input-4-1082ec30e327>:8: SyntaxWarning: name 'train_x' is assigned to before global declaration
  global train_x, test_x, train_y, test_y
<ipython-input-4-1082ec30e327>:8: SyntaxWarning: name 'test_x' is assigned to before global declaration
  global train_x, test_x, train_y, test_y
<ipython-input-4-1082ec30e327>:8: SyntaxWarning: name 'train_y' is assigned to before global declaration
  global train_x, test_x, train_y, test_y
<ipython-input-4-1082ec30e327>:8: SyntaxWarning: name 'test_y' is assigned to before global declaration
  global train_x, test_x, train_y, test_y


In [5]:
load_data()
process_data()

Example Data:
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
Example Labels:
[0 0 0 0 0]


In [6]:
def create_network(model_num):
    model_name = "Model"+str(model_num)
    model_desc = {'name':model_name}
    print(model_name)
    model_name = Sequential()
    num_hidden_layers = randint(0, max_hidden_layers)
    i = 0
    model_name.add(Dense(10, input_shape=(4,), activation='relu', name='input'))
    while i < num_hidden_layers:
        name = 'hidden'+str(i)
        num_neurons = randint(1, max_neurons_per_layer)
        activation = acti_func[randint(0, len(acti_func)-1)]
        model_desc.update({'hidden'+str(i) : (num_neurons, activation)})
        #print(activation)
        model_name.add(Dense(num_neurons, activation=activation, name=name))
        i = i+1
    model_name.add(Dense(3, activation='softmax', name='output'))
    global model_name
    global model_desc

<ipython-input-6-0e67b9f64f4e>:18: SyntaxWarning: name 'model_name' is assigned to before global declaration
  global model_name
<ipython-input-6-0e67b9f64f4e>:19: SyntaxWarning: name 'model_desc' is assigned to before global declaration
  global model_desc


In [7]:
def lr_and_optimizer(model_name):
    learning_rate = random.uniform(min_learning_rate, max_learning_rate)
    #print(learning_rate)
    model_desc.update({'learning_rate' : learning_rate})
    optimizer = Adam(lr=learning_rate)
    model_desc.update({'optimizer' : 'Adam'})
    model_name.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
def print_summary(model_name):
    print('Neural Net Model Summary: ')
    print(model_name.summary())

In [9]:
def train_and_test(model_name):
    model_name.fit(train_x, train_y, verbose = 2, batch_size = 5, epochs = 200)
    results = model_name.evaluate(test_x, test_y)
    global results

<ipython-input-9-915b75c55b69>:4: SyntaxWarning: name 'results' is assigned to before global declaration
  global results


In [10]:
def individual(model_num):
    create_network(model_num)
    #print_summary(model_name)
    lr_and_optimizer(model_name)
    train_and_test(model_name)
    loss_fitness =  (1/float(results[0]))*loss_weight
    acc_fitness = float(results[1])*accuracy_weight
    total_fitness = loss_fitness + acc_fitness
    model_desc.update({'loss_fitness':loss_fitness})
    model_desc.update({'accuracy_fitness':acc_fitness})
    model_desc.update({'total_fitness':total_fitness})
    print(model_name)
    print(model_desc)

In [11]:
def population():
    population_desc = []
    for x in xrange(num_individuals):
        individual(model_num = x)
        population_desc.append(model_desc)
    print(population_desc)
    global population_desc

<ipython-input-11-662326261e74>:7: SyntaxWarning: name 'population_desc' is assigned to before global declaration
  global population_desc


In [12]:
population()

Model0
Epoch 1/200
 - 13s - loss: 0.6291 - acc: 0.6167
Epoch 2/200
 - 0s - loss: 0.1292 - acc: 0.9833
Epoch 3/200
 - 0s - loss: 0.0863 - acc: 0.9667
Epoch 4/200
 - 0s - loss: 0.0753 - acc: 0.9583
Epoch 5/200
 - 0s - loss: 0.4912 - acc: 0.8250
Epoch 6/200
 - 0s - loss: 0.1399 - acc: 0.9583
Epoch 7/200
 - 0s - loss: 0.0633 - acc: 0.9833
Epoch 8/200
 - 0s - loss: 0.1983 - acc: 0.9083
Epoch 9/200
 - 0s - loss: 0.1026 - acc: 0.9667
Epoch 10/200
 - 0s - loss: 0.0699 - acc: 0.9833
Epoch 11/200
 - 0s - loss: 0.0686 - acc: 0.9667
Epoch 12/200
 - 0s - loss: 0.1043 - acc: 0.9583
Epoch 13/200
 - 0s - loss: 0.0975 - acc: 0.9583
Epoch 14/200
 - 0s - loss: 0.1077 - acc: 0.9417
Epoch 15/200
 - 0s - loss: 0.1041 - acc: 0.9750
Epoch 16/200
 - 0s - loss: 0.0681 - acc: 0.9833
Epoch 17/200
 - 0s - loss: 0.0826 - acc: 0.9750
Epoch 18/200
 - 0s - loss: 0.0765 - acc: 0.9667
Epoch 19/200
 - 0s - loss: 0.0957 - acc: 0.9750
Epoch 20/200
 - 0s - loss: 0.0577 - acc: 0.9833
Epoch 21/200
 - 0s - loss: 0.0779 - acc: 

 - 0s - loss: 0.0943 - acc: 0.9583
Epoch 171/200
 - 0s - loss: 0.0348 - acc: 0.9833
Epoch 172/200
 - 0s - loss: 0.1534 - acc: 0.9667
Epoch 173/200
 - 0s - loss: 0.0732 - acc: 0.9833
Epoch 174/200
 - 0s - loss: 0.0824 - acc: 0.9750
Epoch 175/200
 - 0s - loss: 0.1156 - acc: 0.9667
Epoch 176/200
 - 0s - loss: 0.0455 - acc: 0.9833
Epoch 177/200
 - 0s - loss: 0.0663 - acc: 0.9833
Epoch 178/200
 - 0s - loss: 0.0619 - acc: 0.9750
Epoch 179/200
 - 0s - loss: 0.0681 - acc: 0.9750
Epoch 180/200
 - 0s - loss: 0.0886 - acc: 0.9750
Epoch 181/200
 - 0s - loss: 0.0617 - acc: 0.9833
Epoch 182/200
 - 0s - loss: 0.0522 - acc: 0.9833
Epoch 183/200
 - 0s - loss: 0.0469 - acc: 0.9833
Epoch 184/200
 - 0s - loss: 0.0570 - acc: 0.9917
Epoch 185/200
 - 0s - loss: 0.0699 - acc: 0.9750
Epoch 186/200
 - 0s - loss: 0.0730 - acc: 0.9833
Epoch 187/200
 - 0s - loss: 0.0728 - acc: 0.9833
Epoch 188/200
 - 0s - loss: 0.0452 - acc: 0.9833
Epoch 189/200
 - 0s - loss: 0.0523 - acc: 0.9833
Epoch 190/200
 - 0s - loss: 0.0813

 - 0s - loss: 0.4599 - acc: 0.6833
Epoch 132/200
 - 0s - loss: 0.4586 - acc: 0.6833
Epoch 133/200
 - 0s - loss: 0.4610 - acc: 0.6833
Epoch 134/200
 - 0s - loss: 0.4594 - acc: 0.6833
Epoch 135/200
 - 0s - loss: 0.4596 - acc: 0.6833
Epoch 136/200
 - 0s - loss: 0.4586 - acc: 0.6833
Epoch 137/200
 - 0s - loss: 0.4588 - acc: 0.6833
Epoch 138/200
 - 0s - loss: 0.4590 - acc: 0.6833
Epoch 139/200
 - 0s - loss: 0.4595 - acc: 0.6833
Epoch 140/200
 - 0s - loss: 0.4593 - acc: 0.6833
Epoch 141/200
 - 0s - loss: 0.4607 - acc: 0.6833
Epoch 142/200
 - 0s - loss: 0.4598 - acc: 0.6833
Epoch 143/200
 - 0s - loss: 0.4591 - acc: 0.6833
Epoch 144/200
 - 0s - loss: 0.4588 - acc: 0.6833
Epoch 145/200
 - 0s - loss: 0.4586 - acc: 0.6833
Epoch 146/200
 - 0s - loss: 0.4587 - acc: 0.6833
Epoch 147/200
 - 0s - loss: 0.4594 - acc: 0.6833
Epoch 148/200
 - 0s - loss: 0.4595 - acc: 0.6833
Epoch 149/200
 - 0s - loss: 0.4588 - acc: 0.6833
Epoch 150/200
 - 0s - loss: 0.4589 - acc: 0.6833
Epoch 151/200
 - 0s - loss: 0.4609

 - 0s - loss: 0.4584 - acc: 0.6833
Epoch 90/200
 - 0s - loss: 0.4654 - acc: 0.6417
Epoch 91/200
 - 0s - loss: 0.4608 - acc: 0.6750
Epoch 92/200
 - 0s - loss: 0.4632 - acc: 0.6333
Epoch 93/200
 - 0s - loss: 0.4600 - acc: 0.6667
Epoch 94/200
 - 0s - loss: 0.4598 - acc: 0.6583
Epoch 95/200
 - 0s - loss: 0.4664 - acc: 0.6833
Epoch 96/200
 - 0s - loss: 0.4711 - acc: 0.6167
Epoch 97/200
 - 0s - loss: 0.4600 - acc: 0.6833
Epoch 98/200
 - 0s - loss: 0.4639 - acc: 0.6833
Epoch 99/200
 - 0s - loss: 0.4789 - acc: 0.6417
Epoch 100/200
 - 0s - loss: 0.4665 - acc: 0.6833
Epoch 101/200
 - 0s - loss: 0.4609 - acc: 0.6250
Epoch 102/200
 - 0s - loss: 0.4580 - acc: 0.6833
Epoch 103/200
 - 0s - loss: 0.4611 - acc: 0.6833
Epoch 104/200
 - 0s - loss: 0.4635 - acc: 0.6833
Epoch 105/200
 - 0s - loss: 0.4596 - acc: 0.6583
Epoch 106/200
 - 0s - loss: 0.4575 - acc: 0.6917
Epoch 107/200
 - 0s - loss: 0.4596 - acc: 0.6833
Epoch 108/200
 - 0s - loss: 0.4728 - acc: 0.6833
Epoch 109/200
 - 0s - loss: 0.4710 - acc: 0.

 - 0s - loss: 0.2584 - acc: 0.9333
Epoch 48/200
 - 0s - loss: 0.2460 - acc: 0.9583
Epoch 49/200
 - 0s - loss: 0.2439 - acc: 0.9750
Epoch 50/200
 - 0s - loss: 0.2361 - acc: 0.9750
Epoch 51/200
 - 0s - loss: 0.2377 - acc: 0.9583
Epoch 52/200
 - 0s - loss: 0.2299 - acc: 0.9750
Epoch 53/200
 - 0s - loss: 0.2213 - acc: 0.9750
Epoch 54/200
 - 0s - loss: 0.2172 - acc: 0.9750
Epoch 55/200
 - 0s - loss: 0.2119 - acc: 0.9750
Epoch 56/200
 - 0s - loss: 0.2104 - acc: 0.9750
Epoch 57/200
 - 0s - loss: 0.2042 - acc: 0.9833
Epoch 58/200
 - 0s - loss: 0.2006 - acc: 0.9833
Epoch 59/200
 - 0s - loss: 0.1989 - acc: 0.9667
Epoch 60/200
 - 0s - loss: 0.1936 - acc: 0.9667
Epoch 61/200
 - 0s - loss: 0.1888 - acc: 0.9750
Epoch 62/200
 - 0s - loss: 0.1898 - acc: 0.9750
Epoch 63/200
 - 0s - loss: 0.1830 - acc: 0.9750
Epoch 64/200
 - 0s - loss: 0.1804 - acc: 0.9750
Epoch 65/200
 - 0s - loss: 0.1856 - acc: 0.9500
Epoch 66/200
 - 0s - loss: 0.1739 - acc: 0.9750
Epoch 67/200
 - 0s - loss: 0.1698 - acc: 0.9750
Epoch

...

 - 0s - loss: 0.0883 - acc: 0.9667
Epoch 30/200
 - 0s - loss: 0.0699 - acc: 0.9833
Epoch 31/200
 - 0s - loss: 0.0544 - acc: 0.9833
Epoch 32/200
 - 0s - loss: 0.0755 - acc: 0.9750
Epoch 33/200
 - 0s - loss: 0.1536 - acc: 0.9250
Epoch 34/200
 - 0s - loss: 0.1208 - acc: 0.9667
Epoch 35/200
 - 0s - loss: 0.0770 - acc: 0.9667
Epoch 36/200
 - 0s - loss: 0.0856 - acc: 0.9667
Epoch 37/200
 - 0s - loss: 0.0559 - acc: 0.9667
Epoch 38/200
 - 0s - loss: 0.0705 - acc: 0.9583
Epoch 39/200
 - 0s - loss: 0.0505 - acc: 0.9917
Epoch 40/200
 - 0s - loss: 0.0654 - acc: 0.9833
Epoch 41/200
 - 0s - loss: 0.0941 - acc: 0.9667
Epoch 42/200
 - 0s - loss: 0.0876 - acc: 0.9667
Epoch 43/200
 - 0s - loss: 0.0956 - acc: 0.9667
Epoch 44/200
 - 0s - loss: 0.0515 - acc: 0.9833
Epoch 45/200
 - 0s - loss: 0.0642 - acc: 0.9750
Epoch 46/200
 - 0s - loss: 0.0426 - acc: 0.9917
Epoch 47/200
 - 0s - loss: 0.0638 - acc: 0.9917
Epoch 48/200
 - 0s - loss: 0.0558 - acc: 0.9917
Epoch 49/200
 - 0s - loss: 0.0759 - acc: 0.9833
Epoch

Epoch 198/200
 - 0s - loss: 0.0815 - acc: 0.9750
Epoch 199/200
 - 0s - loss: 0.0637 - acc: 0.9667
Epoch 200/200
 - 0s - loss: 0.0596 - acc: 0.9750
30/30 [==============================] - 0s 5ms/step
{'total_fitness': 159.52488708064013, 'hidden0': (3, 'elu'), 'optimizer': 'Adam', 'name': 'Model12', 'learning_rate': 0.009017420178115649, 'loss_fitness': 149.85822069212847, 'accuracy_fitness': 9.666666388511658}
Model13
Epoch 1/200
 - 1s - loss: 1.1458 - acc: 0.3083
Epoch 2/200
 - 0s - loss: 0.7631 - acc: 0.6833
Epoch 3/200
 - 0s - loss: 0.5290 - acc: 0.6333
Epoch 4/200
 - 0s - loss: 0.5180 - acc: 0.6583
Epoch 5/200
 - 0s - loss: 1.4547 - acc: 0.4417
Epoch 6/200
 - 0s - loss: 1.1408 - acc: 0.3917
Epoch 7/200
 - 0s - loss: 1.1207 - acc: 0.3583
Epoch 8/200
 - 0s - loss: 1.1144 - acc: 0.2833
Epoch 9/200
 - 0s - loss: 1.1169 - acc: 0.3250
Epoch 10/200
 - 0s - loss: 1.1093 - acc: 0.3167
Epoch 11/200
 - 0s - loss: 1.1129 - acc: 0.2750
Epoch 12/200
 - 0s - loss: 1.1102 - acc: 0.3167
Epoch 13/2

Epoch 161/200
 - 0s - loss: 1.1033 - acc: 0.2833
Epoch 162/200
 - 0s - loss: 1.1082 - acc: 0.2500
Epoch 163/200
 - 0s - loss: 1.1207 - acc: 0.2667
Epoch 164/200
 - 0s - loss: 1.1113 - acc: 0.2750
Epoch 165/200
 - 0s - loss: 1.1074 - acc: 0.3000
Epoch 166/200
 - 0s - loss: 1.1120 - acc: 0.2917
Epoch 167/200
 - 0s - loss: 1.1121 - acc: 0.3500
Epoch 168/200
 - 0s - loss: 1.1192 - acc: 0.3000
Epoch 169/200
 - 0s - loss: 1.1186 - acc: 0.3083
Epoch 170/200
 - 0s - loss: 1.1084 - acc: 0.3083
Epoch 171/200
 - 0s - loss: 1.1077 - acc: 0.3333
Epoch 172/200
 - 0s - loss: 1.1070 - acc: 0.2667
Epoch 173/200
 - 0s - loss: 1.1144 - acc: 0.2417
Epoch 174/200
 - 0s - loss: 1.1148 - acc: 0.3167
Epoch 175/200
 - 0s - loss: 1.1079 - acc: 0.3417
Epoch 176/200
 - 0s - loss: 1.1081 - acc: 0.3583
Epoch 177/200
 - 0s - loss: 1.1064 - acc: 0.2917
Epoch 178/200
 - 0s - loss: 1.1093 - acc: 0.2833
Epoch 179/200
 - 0s - loss: 1.1129 - acc: 0.2917
Epoch 180/200
 - 0s - loss: 1.1063 - acc: 0.2833
Epoch 181/200
 - 0s 

Epoch 120/200
 - 0s - loss: 0.0512 - acc: 0.9833
Epoch 121/200
 - 0s - loss: 0.0440 - acc: 0.9833
Epoch 122/200
 - 0s - loss: 0.0463 - acc: 0.9917
Epoch 123/200
 - 0s - loss: 0.0532 - acc: 0.9750
Epoch 124/200
 - 0s - loss: 0.0568 - acc: 0.9833
Epoch 125/200
 - 0s - loss: 0.0651 - acc: 0.9833
Epoch 126/200
 - 0s - loss: 0.0734 - acc: 0.9750
Epoch 127/200
 - 0s - loss: 0.0468 - acc: 0.9833
Epoch 128/200
 - 0s - loss: 0.0580 - acc: 0.9833
Epoch 129/200
 - 0s - loss: 0.0389 - acc: 0.9917
Epoch 130/200
 - 0s - loss: 0.1853 - acc: 0.9333
Epoch 131/200
 - 0s - loss: 0.0445 - acc: 0.9917
Epoch 132/200
 - 0s - loss: 0.0567 - acc: 0.9833
Epoch 133/200
 - 0s - loss: 0.0517 - acc: 0.9750
Epoch 134/200
 - 0s - loss: 0.0513 - acc: 0.9833
Epoch 135/200
 - 0s - loss: 0.0679 - acc: 0.9750
Epoch 136/200
 - 0s - loss: 0.0749 - acc: 0.9750
Epoch 137/200
 - 0s - loss: 0.0525 - acc: 0.9833
Epoch 138/200
 - 0s - loss: 0.0519 - acc: 0.9833
Epoch 139/200
 - 0s - loss: 0.0511 - acc: 0.9833
Epoch 140/200
 - 0s 

Epoch 82/200
 - 0s - loss: 0.4720 - acc: 0.6250
Epoch 83/200
 - 0s - loss: 0.4610 - acc: 0.6417
Epoch 84/200
 - 0s - loss: 0.4664 - acc: 0.5833
Epoch 85/200
 - 0s - loss: 0.4599 - acc: 0.6833
Epoch 86/200
 - 0s - loss: 0.4601 - acc: 0.6833
Epoch 87/200
 - 0s - loss: 0.4781 - acc: 0.6667
Epoch 88/200
 - 0s - loss: 0.4708 - acc: 0.6833
Epoch 89/200
 - 0s - loss: 0.4576 - acc: 0.6833
Epoch 90/200
 - 0s - loss: 0.4645 - acc: 0.6417
Epoch 91/200
 - 0s - loss: 0.4754 - acc: 0.6500
Epoch 92/200
 - 0s - loss: 0.4602 - acc: 0.6583
Epoch 93/200
 - 0s - loss: 0.4593 - acc: 0.6833
Epoch 94/200
 - 0s - loss: 0.4729 - acc: 0.6500
Epoch 95/200
 - 0s - loss: 0.4669 - acc: 0.6583
Epoch 96/200
 - 0s - loss: 0.4592 - acc: 0.6667
Epoch 97/200
 - 0s - loss: 0.4598 - acc: 0.6667
Epoch 98/200
 - 0s - loss: 0.4605 - acc: 0.6417
Epoch 99/200
 - 0s - loss: 0.4577 - acc: 0.6833
Epoch 100/200
 - 0s - loss: 0.4628 - acc: 0.5917
Epoch 101/200
 - 0s - loss: 0.4618 - acc: 0.6417
Epoch 102/200
 - 0s - loss: 0.4675 - a

In [13]:
def grade():    
    fitness_sum = 0
    'Find average fitness for a population.'
    for x in population_desc:
        fitness_sum = x.get("total_fitness") + fitness_sum
    avg_fitness = fitness_sum/len(population_desc)
    print(avg_fitness)
    global avg_fitness

<ipython-input-13-e0a3493f5de1>:8: SyntaxWarning: name 'avg_fitness' is assigned to before global declaration
  global avg_fitness


In [14]:
grade()

80.7824774107


In [37]:
from operator import itemgetter
parents = []
for x in population_desc:
    if x.get("total_fitness") >= avg_fitness:
        parents.append((x.get('name'), x.get('total_fitness')))
        difference = x.get('total_fitness') - avg_fitness
        #print(x.get("name")+"    "+str(x.get("total_fitness"))+"    "+str(difference))
def get_key(item):
    return item[1]
parents_sorted = sorted(parents, key = get_key)
print(parents_sorted)
if len(parents) % 2 !=0:
    print('odd number')
        

[('Model7', 133.08223815090759), ('Model14', 142.31747003551183), ('Model5', 149.161715280161), ('Model12', 159.52488708064013), ('Model8', 176.48972828362488), ('Model6', 178.09340638684392)]


In [104]:
parent1 = (item for item in population_desc if item["name"] == "Model7").next()
parent2 = (item for item in population_desc if item["name"] == "Model14").next()
print(parent1)
print(parent2)
new_learning_rate = (parent1.get("learning_rate")+parent2.get("learning_rate"))/2
print(new_learning_rate)
def parent_select():
    num = randint(0,1)
    if num == 0:
        return parent1
    else:
        return parent2
hidden0 = parent_select().get('hidden0')
hidden1 = parent_select().get('hidden1')
print(hidden0)
print(hidden1)

{'hidden3': (66, 'softsign'), 'hidden2': (53, 'softsign'), 'hidden1': (24, 'elu'), 'hidden0': (5, 'relu'), 'optimizer': 'Adam', 'name': 'Model7', 'hidden4': (64, 'relu'), 'learning_rate': 0.0028422883290202027, 'loss_fitness': 123.74890477783782, 'accuracy_fitness': 9.333333373069763, 'total_fitness': 133.08223815090759}
{'hidden1': (42, 'linear'), 'hidden0': (42, 'relu'), 'optimizer': 'Adam', 'name': 'Model14', 'learning_rate': 0.0024605493806398467, 'loss_fitness': 132.65080364700017, 'total_fitness': 142.31747003551183, 'accuracy_fitness': 9.666666388511658}
0.00265141885483
(42, 'relu')
(24, 'elu')
